In [1]:
# Install the packages
!pip install tabpfn
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, roc_curve
from tabpfn import TabPFNClassifier
import pickle
from imblearn.over_sampling import SMOTE


DATA_PATH = "/content/drive/MyDrive/binary_classificaction_smote/classification_input_binary.csv"
MODEL_PATH = "/content/drive/MyDrive/binary_classificaction_smote/tabpfn_classifier_2class.pkl"
FIANL_RESULTS_PATH = "/content/drive/MyDrive/binary_classificaction_smote/test_result_TabPFN_binary_classification.csv"

# Load data
data = pd.read_csv(DATA_PATH)
X = data.iloc[:, 3:]
y = data.iloc[:, 0]

# Initialize performance metrics storage
train_performance = []
test_performance = []
all_test_preds = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
round_id = 1

smote = SMOTE(random_state=42)

# Loop through each fold
for train_index, test_index in skf.split(X, y):
    print(f"\nRound {round_id}")
    round_id += 1

    # Split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Train model
    model = TabPFNClassifier(device="cuda")
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_proba_test = model.predict_proba(X_test)[:, 1]  # Get predicted probabilities for the positive class
    y_proba_train = model.predict_proba(X_train)[:, 1]
    train_auc = roc_auc_score(y_train, y_proba_train)
    test_auc = roc_auc_score(y_test, y_proba_test)  # Get AUC for the positive class

    # Calculate accuracy and F1 score
    train_acc = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train, average='weighted')

    test_acc = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted')

    train_performance.append({'Accuracy': train_acc, 'F1': train_f1, 'AUC': train_auc})
    test_performance.append({'Accuracy': test_acc, 'F1': test_f1, 'AUC': test_auc})

    print("Train Accuracy:", round(train_acc, 3), "F1:", round(train_f1, 3))
    print("Test  Accuracy:", round(test_acc, 3), "F1:", round(test_f1, 3))

    # Store test predictions
    fold_result = pd.DataFrame({
        'true_label': y_test.values,
        'predicted_label': y_pred_test,
        'predicted_proba': y_proba_test,
        'group': data.iloc[test_index]['group'].values
    })
    all_test_preds.append(fold_result)


Round 1
Train Accuracy: 0.993 F1: 0.993
Test  Accuracy: 0.852 F1: 0.845

Round 2
Train Accuracy: 0.993 F1: 0.993
Test  Accuracy: 0.869 F1: 0.853

Round 3
Train Accuracy: 0.99 F1: 0.99
Test  Accuracy: 0.836 F1: 0.831

Round 4
Train Accuracy: 0.994 F1: 0.994
Test  Accuracy: 0.852 F1: 0.85

Round 5
Train Accuracy: 0.998 F1: 0.998
Test  Accuracy: 0.869 F1: 0.853

Round 6
Train Accuracy: 0.995 F1: 0.995
Test  Accuracy: 0.934 F1: 0.936

Round 7
Train Accuracy: 0.994 F1: 0.994
Test  Accuracy: 0.869 F1: 0.86

Round 8
Train Accuracy: 0.995 F1: 0.995
Test  Accuracy: 0.85 F1: 0.836

Round 9
Train Accuracy: 0.988 F1: 0.988
Test  Accuracy: 0.9 F1: 0.893

Round 10
Train Accuracy: 0.994 F1: 0.994
Test  Accuracy: 0.883 F1: 0.872


In [6]:
# Save final results
final_result = pd.concat(all_test_preds, ignore_index=True)
final_result.to_csv(FIANL_RESULTS_PATH, index=False)

df_train_perf = pd.DataFrame(train_performance)
df_test_perf = pd.DataFrame(test_performance)

print("\nAverage Train Performance:")
for metric in df_train_perf.columns:
    print(f"{metric:<10}: {df_train_perf[metric].mean():.3f} ± {df_train_perf[metric].std():.3f}")

print("\nAverage Test Performance:")
for metric in df_test_perf.columns:
    print(f"{metric:<10}: {df_test_perf[metric].mean():.3f} ± {df_test_perf[metric].std():.3f}")


Average Train Performance:
Accuracy  : 0.993 ± 0.003
F1        : 0.993 ± 0.003
AUC       : 1.000 ± 0.000

Average Test Performance:
Accuracy  : 0.872 ± 0.029
F1        : 0.863 ± 0.031
AUC       : 0.893 ± 0.045


In [5]:
# Train final model on entire dataset
MODEL_PATH = "/content/drive/MyDrive/binary_classificaction_smote/tabpfn_classifier_2class.pkl"
final_model = TabPFNClassifier(device="cuda")
final_model.fit(X, y)

# Save the final model
with open(MODEL_PATH, "wb") as f:
    pickle.dump(final_model, f)